In [6]:
import pandas as pd
df = pd.read_csv("../data/ufo_sightings.csv")
# Display the DataFrame
print(df)

              date_time             city_area state country ufo_shape  \
0      10/10/1949 20:30            san marcos    tx      us  cylinder   
1      10/10/1949 21:00          lackland afb    tx     NaN     light   
2      10/10/1955 17:00  chester (uk/england)   NaN      gb    circle   
3      10/10/1956 21:00                  edna    tx      us    circle   
4      10/10/1960 20:00               kaneohe    hi      us     light   
...                 ...                   ...   ...     ...       ...   
80327    9/9/2013 21:15             nashville    tn      us     light   
80328    9/9/2013 22:00                 boise    id      us    circle   
80329    9/9/2013 22:00                  napa    ca      us     other   
80330    9/9/2013 22:20                vienna    va      us    circle   
80331    9/9/2013 23:00                edmond    ok      us     cigar   

       encounter_length described_encounter_length  \
0                2700.0                 45 minutes   
1              

In [8]:
df_new = df.dropna()
print(df_new)
df_new.isnull().values.any()


              date_time   city_area state country ufo_shape  encounter_length  \
0      10/10/1949 20:30  san marcos    tx      us  cylinder            2700.0   
3      10/10/1956 21:00        edna    tx      us    circle              20.0   
4      10/10/1960 20:00     kaneohe    hi      us     light             900.0   
5      10/10/1961 19:00     bristol    tn      us    sphere             300.0   
7      10/10/1965 23:45     norwalk    ct      us      disk            1200.0   
...                 ...         ...   ...     ...       ...               ...   
80327    9/9/2013 21:15   nashville    tn      us     light             600.0   
80328    9/9/2013 22:00       boise    id      us    circle            1200.0   
80329    9/9/2013 22:00        napa    ca      us     other            1200.0   
80330    9/9/2013 22:20      vienna    va      us    circle               5.0   
80331    9/9/2013 23:00      edmond    ok      us     cigar            1020.0   

      described_encounter_l

False

In [9]:
df_new.to_csv('../data/ufo_sightings_NMV.csv', index=False) # saving it as No Missing Values (MSV)